In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data/small_corpus.csv',index_col=0)
df['reviews']= df['reviews'].astype(str)

In [4]:
df.head()

,ratings,reviews
122468,1,Recently UBISOFT had to settle a huge class-ac...
495370,1,"code didn't work, got me a refund."
214717,1,"these do not work at all, all i get is static ..."
231734,1,well let me start by saying that when i first ...
154709,1,"Dont waste your money, you will just end up us..."


In [5]:
def score_to_Target(value):
    if value >= 5:
        return 2
    if value <= 4 and value >= 2:
        return 1
    else:
        return 0

In [6]:
df['labels'] = df['ratings'].apply(lambda x:score_to_Target(x))

In [7]:
from sklearn.utils import shuffle
df = shuffle(df)

In [8]:
df.head()

,ratings,reviews,labels
271444,5,ONE OF THE BEST FIGHTERS GAMES,2
249366,2,The game only gets a 2/5 star rating from me.\...,1
487809,3,Arrived on time and was in good condition howe...,1
256742,1,I don't like it because no one can hear me bea...,0
403279,5,"Amazing. I could write a long review here, but...",2


In [9]:
import re
import string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [10]:
df['text'] = df['reviews'].apply(lambda x:clean_text(x))

In [11]:
df.head()

,ratings,reviews,labels,text
271444,5,ONE OF THE BEST FIGHTERS GAMES,2,one of the best fighters games
249366,2,The game only gets a 2/5 star rating from me.\...,1,the game only gets a star rating from methe g...
487809,3,Arrived on time and was in good condition howe...,1,arrived on time and was in good condition howe...
256742,1,I don't like it because no one can hear me bea...,0,i dont like it because no one can hear me beac...
403279,5,"Amazing. I could write a long review here, but...",2,amazing i could write a long review here but i...


In [12]:
!pip install simpletransformers -q

In [13]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

In [14]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [15]:
model_args = ClassificationArgs(num_train_epochs=1)

In [16]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df,
                                     stratify=df["labels"],
                                     random_state=42)

In [17]:
SAVE_EVERY_N_EPOCHS = 3
import math

In [18]:
steps_per_epoch = math.floor(len(train_df) / SAVE_EVERY_N_EPOCHS)
if(len(train_df) % SAVE_EVERY_N_EPOCHS > 0):
    steps_per_epoch +=1
import sklearn
model_args = ClassificationArgs(num_train_epochs=10)
model_args.max_seq_length=512
model_args.sliding_window=True
model_args.overwrite_output_dir = True
model_args.evaluate_during_training = True
model_args.eval_batch_size = 20
model_args.train_batch_size =20
model_args.output_dir = 'output/'
model_args.best_model_dir = 'bestmodel/'
model_args.use_cached_eval_features = True
model_args.use_early_stopping = True
model_args.early_stopping_metric = 'eval_loss'
model_args.early_stopping_metric_minimize = True
model_args.save_steps = steps_per_epoch
model_args.save_model_every_epoch = False

In [19]:
model = ClassificationModel(
    "albert", "albert-base-v2", args=model_args,num_labels=3
)

In [20]:
model.train_model(train_df, eval_df=test_df,accuracy = sklearn.metrics.accuracy_score)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


INFO:simpletransformers.classification.classification_model: 3724 features created from 3375 samples.


RuntimeError: CUDA out of memory. Tried to allocate 120.00 MiB (GPU 0; 15.75 GiB total capacity; 14.87 GiB already allocated; 51.31 MiB free; 14.91 GiB reserved in total by PyTorch)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_df,acc=sklearn.metrics.accuracy_score)